In [3]:
%run jupyter/notebook_modules.ipynb

In [10]:
import ops.kaggle_dependencies

In [15]:
import os

import kaggle
from google.cloud import bigquery

import bq
from competition import THIS_COMPETITION

In [12]:
TMP_LOCATION = "/tmp"

In [36]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition=bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

In [ ]:
def upload(source_filename, table_ref):
    with open(source_filename, 'rb') as source_file:
        job = bq.client.load_table_from_file(
            source_file,
            table_ref,
            location='US',
            job_config=job_config)
        return job

In [35]:
def initialize(truncate=False):
    dataset = bq.client.create_dataset(bq.dataset_id, exists_ok=True)
    files_tables = ((str(f), os.path.splitext(str(f))[0])
                    for f in kaggle.api.competition_list_files(THIS_COMPETITION))
    if not truncate:
        existing_tables = set(t.table_id for t in bq.client.list_tables(dataset))
        files_tables = ((f, table)
                        for f, table in files_tables
                        if table not in existing_tables)
    # synchonously upload competition files
    for f, table in files_tables:
        kaggle.api.competition_download_file(THIS_COMPETITION, f, TMP_LOCATION)
        path = os.path.join(TMP_LOCATION, f)
        z = "{path}.zip".format(path=path)
        !unzip -u $z
        !chmod +r $f
        os.remove(z)
        table_ref = dataset.table(table)
        job = upload(f, table_ref)
        job.result() # wait for completion (make uploads synchronous)
        os.remove(f)